## Exploratory Data Analysis (EDA) and Data Cleaning

### EDA

The data we obtained from XXXX came in the form of XXXX many .wav files of varying lengths, with an accompanying metadata .csv file containing the following information:
- Biological classification (genus, species).
- Recording information (recordist, date, apparatus used, geographical location, background species, etc.).
(The `librosa` package for audio processing and visualisation is used throughout our work. See https://librosa.org/doc/latest/index.html# for more information.)

In [1]:
import librosa, librosa.display 
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
import math

Many of the files contained a human speech component at the beginning (the recordist or scientist stating the catalogue number of the recording, for instance). We used the following code to extract the second non-silent chunk of each audio file, which typically contained a useful section of the insect sound recording.

In [2]:
def remove_Talking(f):
    chirp_song, sr = librosa.load(f)
    split=librosa.effects.split(chirp_song, top_db=60)
    chirp_song_split=chirp_song[split[1,0]:split[1,1]]
    if chirp_song_split.shape[0] > 220500:
        chirp_song_split = chirp_song_split[:220500]
    return chirp_song_split

Further EDA revealed that even after the initial human speech component was eliminated, some audio files still contained human speech. The following code located these audio files and removed them from our analysis.

In [ ]:
SAMPLING_RATE = 16000

import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=False)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

SAMPLING_RATE=16000
wav = read_audio('E://Chirp_Files/117596.wav', sampling_rate=SAMPLING_RATE)
# get speech timestamps from full audio file
speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=SAMPLING_RATE)
pprint(speech_timestamps)

import sys
sys.path.append('E://Chirp_Files')

import os
# assign directory
directory = 'E://Chirp_Files'
no_voice_files=[]
voice_files=[]

# iterate over files in
# that directory
n = 0
m = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        n+=1
        m+=1
        wav = read_audio(f, sampling_rate=SAMPLING_RATE)
        speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=SAMPLING_RATE)
        if speech_timestamps == []:
            no_voice_files.append(f)
        else:
            voice_files.append(f)
        if n % 100 == 0:
            print(m)
            n = 0
            df = pd.DataFrame(voice_files)
            df.to_csv("voice_files.csv", header=False, index=False)
            df = pd.DataFrame(no_voice_files)
            df.to_csv("no_voice_files.csv", header=False, index=False)

Out of the 12602 audio files remaining after `remove_Talking` was applied, this reduced the size of our data set to 6122.

## Data Cleaning

The next step was to clean the audio of any further background noise. Since crickets, cicadas, and katydids typically chirp at frequencies in the 2-10kHz range, we used the following code to remove frequencies below 800Hz from the audio.

In [ ]:
def remove_low_freq(song):
    D = np.abs(librosa.stft(song, n_fft=n_fft,  hop_length=hop_length))
    #FIND OUT HOW MANY FREQUENCIES WE REMOVED HERE!
    D=D[64:]
    return D